# Day 22, Part 2

If you have something like this (which we _will_ have):

$$f(i) = Ai + b \pmod{L}$$

then, (and first), if `A` is super large, we can simplify it by doing `a = A % L` and we've maintained equivalance with the original expression, but now we have:

$$f(i) = ai + b \pmod{L}$$

next, we'll have to _exponentiate_ our function, that is...

$$f^2(i) = ( f \circ f \)(i) = ci + d \pmod{L}$$

but values do we need for `c` and `d` above? Turns out they are:

$$c = a^2 \pmod{L}$$

$$d = ab + b \pmod{L}$$

thus we have a way to exponentiate $f(i)$, which means we can do clever things to exponentiate $f(i)$ to very high exponents (via clasic "Binary Exponentiation"). That's what this notebooks will do to solve day 22, part 2.

### Define `f(i)`

We'll use SymPy to help us find $f(i)$, and use the shuffling routines I wrote in `s.py`.

In [1]:
from sympy import *

init_printing()

In [2]:
from s import get_card, steps

length = 119315717514047
iters = 101741582076661     # <-- this will become our exponent later to exponentiate `f(i)`

In [3]:
# The final index we're interested in knowing about, as a SymPy variable:
i = var('i')

# The original position of the card which ends up at `i` after shuffling (Python ducktyping to save the day here):
original_i = get_card(i, length, steps, on_demand_modulo=False)

# Setting `on_demand_modulo=False` above sort-of lets us "factor out" the modulo operation, so now we apply it now:
original_i = (original_i % length)

print(original_i)

Mod(115454980177290844567281738476683890138352496246839940173996738389070217353588176067092747921376730583751094263579593756541946110963522306094901859879172082061429733011953956417482330770019329550072601913647817006548956550080296079927361292186890392098322236033780459966491440226045131629980768457580137578711104834425214142283809372297302789453005622927278599317793530290809322053336212402080838566927239199863558620560053738873436254088230179015143992258229038445957814576795732011703908083840025223348729145273685972238971772456717710860479949410035302400000000000*i + 55921941268876, 119315717514047)


In [4]:
# !!!! NOT SURE HOW TO EXTRACT THINGS FROM THE SYMPY EXPRESSION ABOVE, SO I JUST COPY/PASTED THE VALUES BELOW:

a = 115454980177290844567281738476683890138352496246839940173996738389070217353588176067092747921376730583751094263579593756541946110963522306094901859879172082061429733011953956417482330770019329550072601913647817006548956550080296079927361292186890392098322236033780459966491440226045131629980768457580137578711104834425214142283809372297302789453005622927278599317793530290809322053336212402080838566927239199863558620560053738873436254088230179015143992258229038445957814576795732011703908083840025223348729145273685972238971772456717710860479949410035302400000000000
b = 55921941268876

a = a % length
b = b % length

a, b

We did it. We know `f(i)` now (we've computed both `a` and `b`).

### Binary Exponentiation

https://en.wikipedia.org/wiki/Exponentiation_by_squaring

First a few examples to show the idea (these also let me check the correctness of the approach because I compared these values with those I get by just calling `get_card()` over-and-over and seeing that I get the same results). Here is the idea:

In [5]:
a_squared = (a * a) % length
b_squared = ((a * b) + b) % length

a_squared, b_squared

In [6]:
a_cubed = (a_squared * a) % length
b_cubed = ((a_squared * b) + b_squared) % length

a_cubed, b_cubed

In [7]:
a_fourth = (a_squared * a_squared) % length
b_fourth = ((a_squared * b_squared) + b_squared) % length

a_fourth, b_fourth

In [8]:
a_fifth = (a_cubed * a_squared) % length
b_fifth = ((a_cubed * b_squared) + b_cubed) % length

a_fifth, b_fifth

In [9]:
a_seventh = (a_fourth * a_cubed) % length
b_seventh = ((a_fourth * b_cubed) + b_fourth) % length

a_seventh, b_seventh

In [10]:
a_eighth = (a_fourth * a_fourth) % length
b_eighth = ((a_fourth * b_fourth) + b_fourth) % length

a_eighth, b_eighth

In [11]:
a_15th = (a_eighth * a_seventh) % length
b_15th = ((a_eighth * b_seventh) + b_eighth) % length

a_15th, b_15th

Now, let's do it for real:

In [12]:
bit = 0

a_exp = a
b_exp = b

a_answer = None
b_answer = None

while (1 << bit) <= iters:
    if (1 << bit) & iters:
        # This bit is set.
        if a_answer is None:
            a_answer = a_exp
            b_answer = b_exp
        else:
            a_next = (a_exp * a_answer) % length
            b_next = ((a_exp * b_answer) + b_exp) % length
            a_answer = a_next
            b_answer = b_next
    
    a_next = (a_exp * a_exp) % length
    b_next = ((a_exp * b_exp) + b_exp) % length
    a_exp = a_next
    b_exp = b_next
    
    bit += 1
    
a_answer, b_answer

### Answer the challenge's question, finally!

In [13]:
i = 2020

f_answer = ((a_answer * i) + b_answer) % length

f_answer